In [ ]:
!pip install transformers
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd

train_df = pd.read_csv('non_skewed_train.csv')
test_df = pd.read_csv('non_skewed_test.csv')
df = pd.concat([train_df, test_df])
df

,id,text,label
0,86426.0,<user> ask native americans take,1
1,90194.0,<user> <user> go home drunk <user> maga trump2...,1
2,16820.0,amazon investigating chinese employees selling...,0
3,62688.0,<user> someone vetaken piece shit volcano <emo...,1
4,97670.0,<user> liberals kookoo,1
...,...,...,...
3531,NaN,eagles fucking trash,1
3532,78174.0,<user> <user> fraud amp fight anti semitism ex...,1
3533,95459.0,<user> thank jourdan amp hayley <emoji : face_...,0
3534,64878.0,<user> <user> fake news term used rather word ...,0


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

,id,text,label
0,NaN,hoes lying fat hiding backpage profile,1
1,NaN,<user> <user> outchea bitch,1
2,51853.0,<user> spoke migrant found member islamic stat...,0
3,32872.0,<user> <user> <user> <user> <user> <user> funn...,0
4,78319.0,<user> oh love much idea daughter family strug...,0
...,...,...,...
17675,98615.0,<user> <user> wait gun control oh wait hoggy b...,0
17676,74479.0,<user> bitter obama seriously worried obama he...,1
17677,61200.0,<user> bono attempt inculcate masses later lif...,1
17678,36197.0,<user> looks like selling sole hater liberals ...,0


In [ ]:
test_df = df.sample(frac = 0.2, random_state = 42)
train_df = df.drop(test_df.index)

In [ ]:
train_df

,id,text,label
1,NaN,<user> <user> outchea bitch,1
2,51853.0,<user> spoke migrant found member islamic stat...,0
4,78319.0,<user> oh love much idea daughter family strug...,0
6,19775.0,<user> bit thanks concern competitor wants wor...,0
7,37659.0,<user> <user> <user> clearly right flaws doo t...,0
...,...,...,...
17674,NaN,bitch putting long lashes like 3abde,1
17675,98615.0,<user> <user> wait gun control oh wait hoggy b...,0
17676,74479.0,<user> bitter obama seriously worried obama he...,1
17677,61200.0,<user> bono attempt inculcate masses later lif...,1


In [ ]:
train_df.to_csv('non_skewed_train.csv', index = False)
test_df.to_csv('non_skewed_test.csv', index = False)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'non_skewed_train.csv', 'test': 'non_skewed_test.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: [' id', 'text', 'label'],
        num_rows: 14144
    })
    test: Dataset({
        features: [' id', 'text', 'label'],
        num_rows: 3536
    })
})

In [ ]:
dataset = dataset.remove_columns([' id'])

In [ ]:
dataset['train'][0]

{'text': '<user> <user> outchea bitch', 'label': 1}

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length = 30)
  return encoding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/14144 [00:00<?, ? examples/s]

Map:   0%|          | 0/3536 [00:00<?, ? examples/s]

In [ ]:
train_ds = encoded_dataset["train"].shuffle(seed=42)
val_ds = encoded_dataset["test"].shuffle(seed=42)

In [ ]:
train_ds.set_format("torch")
val_ds.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased",
                                                           num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import accelerate

accelerate.__version__

'0.27.2'

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"distilbert-olid",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjusted batch size
    per_device_eval_batch_size=8,  # Adjusted batch size
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    gradient_accumulation_steps=2,  # Gradient accumulation
    logging_steps=100,  # Adjust as needed
    eval_steps=200,  # Adjust as needed
)

In [ ]:
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AdamW, get_scheduler
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
scheduler = get_scheduler(
    "linear",
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_ds) // args.gradient_accumulation_steps * args.num_train_epochs,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
      logits = logits[0]
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(predictions, labels)
    f1 = f1_score(predictions, labels, average = 'macro')

    return {"accuracy": acc, "f1": f1}

In [ ]:
train_ds

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 43516
})

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.370500,0.351158,0.832014,0.832007
2,0.325500,0.357464,0.840498,0.839683
3,0.237000,0.410016,0.833145,0.833119
4,0.190800,0.635084,0.837387,0.836921
5,0.103900,0.902078,0.814762,0.814186


TrainOutput(global_step=4420, training_loss=0.2494282520734347, metrics={'train_runtime': 385.8758, 'train_samples_per_second': 183.271, 'train_steps_per_second': 11.454, 'total_flos': 548911783180800.0, 'train_loss': 0.2494282520734347, 'epoch': 5.0})

In [ ]:
preds = trainer.predict(val_ds)

In [ ]:
val_ds[0]

{'text': '1 man seriously needs mental health treatment firearms confiscated longer danger others 2 waiting gun control advocates add list school shootings 3 buys 22 caliber handguns url',
 'label': tensor(0),
 'input_ids': tensor([  101,  1015,  2158,  5667,  3791,  5177,  2740,  3949, 13780, 17182,
          2936,  5473,  2500,  1016,  3403,  3282,  2491, 13010,  5587,  2862,
          2082,  5008,  2015,  1017, 23311,  2570, 15977, 28497,  2015,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1])}

In [ ]:
preds

PredictionOutput(predictions=array([[ 0.52272296, -0.66745377],
       [ 1.3912488 , -1.4710138 ],
       [-2.0463636 ,  2.4409108 ],
       ...,
       [ 0.8247807 , -0.939085  ],
       [ 1.4602128 , -1.564892  ],
       [-3.0135646 ,  3.5190675 ]], dtype=float32), label_ids=array([0, 0, 1, ..., 0, 0, 1]), metrics={'test_loss': 0.3574635088443756, 'test_accuracy': 0.8404977375565611, 'test_f1': 0.8396834931002062, 'test_runtime': 4.4599, 'test_samples_per_second': 792.851, 'test_steps_per_second': 99.106})

In [ ]:
pred_vals = np.argmax(preds.predictions, axis=-1)
labels = preds.label_ids
pred_vals, labels

(array([0, 0, 1, ..., 0, 0, 1]), array([0, 0, 1, ..., 0, 0, 1]))

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
print('roc: ', roc_auc_score(labels, pred_vals))
print('f1: ', f1_score(labels, pred_vals, average = 'macro'))
print('accuracy: ', accuracy_score(labels, pred_vals))

roc:  0.8403792290970514
f1:  0.8396834931002062
accuracy:  0.8404977375565611


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
trainer.push_to_hub('distilbert-olid')

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

events.out.tfevents.1709653530.fd2d11be4705.544.0:   0%|          | 0.00/15.9k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ARC4N3/distilbert-olid/commit/76ae25bca01652520c58f928413fb49804b5fce7', commit_message='distilbert-olid', commit_description='', oid='76ae25bca01652520c58f928413fb49804b5fce7', pr_url=None, pr_revision=None, pr_num=None)